In [28]:
# !pip3 install google 
# !pip3 install beautifulsoup4
# !pip3 install -U sentence-transformers
# !pip3 install umap-learn
# !pip3 install hdbscan
# !pip3 install kaggle
# !pip install bertopic
# !pip3 install sentence_splitter
!pip install spacy-langdetect
!pip install language-detector
!pip install symspellpy
!pip install sentence-transformers

     |████████████████████████████████| 1.0MB 4.7MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993461 sha256=d1c210f881ef79a484d351e4c293dc30f40bbee94755f41fec8c4cc3d17066f5
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect
  Created wheel for language-detector: filename=language_detector-5.0.2-cp36-none-any.whl size=7055 sha256=81e9965f3e50d19986a26f95624d3bed26e0f051684b4e94c7559d9c9cca039e
  Stored in directory: /root/.cache/pip/wheels/7d/37/fa/2098a4aa6c0d94d6ddff0d3a79669e12bc4f7baca8a760b3db
Successfully built language-detector
     |████████████████████████████████| 2.6MB 4.6MB/s 


In [3]:
import numpy as np
import pandas as pd
from googlesearch import search 
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer, util
import hdbscan
import umap
import matplotlib.pyplot as plt
from bertopic import BERTopic
from sentence_splitter import SentenceSplitter, split_text_into_sentences
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
#Get Search Term from User
def get_pages_link(term):
  link_list = []
  c_list = []
  for j in search(term, tld="co.in", num=20, stop=10, pause=2):
    link_list.append(j)
  # print(link_list)
  return link_list

In [5]:
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [6]:
#Function to get Input Query and fetch data from Google
def get_data(text):
  pages_link_data = get_pages_link(text)
  complete_text_data = []
  for link_name in pages_link_data:
    html = urlopen(link_name).read()
    soup = BeautifulSoup(html, features="html.parser")
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    complete_text_data.append(text)
  return complete_text_data

#It breaks if there is 403 Page, needs exception there

In [7]:
#Text Preprocessing to remove useless Data and only keep sentences
def replaceABwithC(input, pattern, replaceWith): 
    return input.replace(pattern, replaceWith) 

In [8]:
def text_cleaning(data):
  n_data = []
  for item in data:
    # pattern = '\n'
    # replaceWith = ' '
    # n_data.append(item.replace("\n", "  "))
    # n_data.append(item.replace("\xa0", "  "))
    nitem = item.replace("\n", "  ")
    n_data.append(nitem.replace("\xa0", "  "))
    # print(replaceABwithC(item,pattern,replaceWith) )
  return n_data

In [82]:
#Creating Dataframe for data
def create_df(n_data):
  df = pd.DataFrame(n_data, columns=["data"]) 
  return df

In [83]:
#Get Dataframe stats
def get_df_stats(df):
  print(df.shape)
  print(df.columns)

In [84]:
#Dumping CSV for Dataframe
def df_to_csv(df):
  df.to_csv("df.csv")

In [85]:
# Load Bert model 
def bert_embedding_transformation(df):
  model = SentenceTransformer('bert-base-nli-mean-tokens')
  sentences=[]
  for i in df.data:
    sentences.append(i)
  sentence_embeddings = model.encode(sentences)
  return sentence_embeddings

In [86]:
# Use K-means to get centroid and make a cluster out of it
def create_text_clusters(embeddings):
  num_clusters = 5
  km = KMeans(n_clusters=num_clusters)
  km.fit(embeddings)
  labels=km.labels_.tolist()
  df["Cluster_Label"]=labels
  return df

In [87]:
#Calling Functions
data = get_data("America")
da = text_cleaning(data)
df = create_df(da)
get_df_stats(df)
df_to_csv(df)
semb = bert_embedding_transformation(df)
cluster_df = create_text_clusters(semb)

(10, 1)
Index(['data'], dtype='object')


In [92]:
splitter = SentenceSplitter(language='en')

In [93]:
ml = []
for item in da:
  res = splitter.split(text=item)
  ml.append(res)


In [94]:
final_list  = []
for item in ml:
  for it in item:
    final_list.append(it)
print(len(final_list))

4887


In [95]:
final_list

['United States - Wikipedia United States From Wikipedia, the free encyclopedia Jump to navigation Jump to search Country located primarily in North America "America", "US", "USA", and "United States of America" redirect here.',
 'For the landmass comprising North, Central, South America, and the Caribbean, see Americas.',
 'For other uses, see America (disambiguation), US (disambiguation), USA (disambiguation), and United States (disambiguation).',
 'Coordinates: 40°N 100°W\ufeff / \ufeff40°N 100°W\ufeff / 40; -100 United States of America Flag Coat of arms Motto: "In God We Trust"[1] Other traditional mottos: "E pluribus unum" (Latin)[2]"Out of many, one" "Annuit cœptis" (Latin)[2]"He has favored our undertakings" "Novus ordo seclorum" (Latin)[2]"New order of the ages" Anthem: "The Star-Spangled Banner"[3] March: "The Stars and Stripes Forever"[4][5] Great Seal: [2]Show globe (states and D.C. only)Show the U.S. and its territoriesCapital Washington, D.C. 38°53′N 77°01′W\ufeff / \ufef

In [96]:
model = BERTopic("distilbert-base-nli-mean-tokens", verbose=True)
topics, probabilities = model.fit_transform(final_list)

2020-11-08 09:52:40,144 - BERTopic - Loaded BERT model
INFO:BERTopic:Loaded BERT model
2020-11-08 09:52:46,699 - BERTopic - Transformed documents to Embeddings
INFO:BERTopic:Transformed documents to Embeddings
2020-11-08 09:52:59,605 - BERTopic - Reduced dimensionality with UMAP
INFO:BERTopic:Reduced dimensionality with UMAP
2020-11-08 09:53:00,326 - BERTopic - Clustered UMAP embeddings with HDBSCAN
INFO:BERTopic:Clustered UMAP embeddings with HDBSCAN


In [102]:
 model.get_topic(5)

[('department', 0.40287955023988725),
 ('state', 0.2259851302226848),
 ('interior', 0.12975682144387582),
 ('united', 0.12716990966432629),
 ('washington', 0.12418400605044079),
 ('statistics', 0.1180005651916602),
 ('states', 0.11796307404311573),
 ('house', 0.09539455191675854),
 ('bureau', 0.08985003579345045),
 ('labor', 0.0873613544458204),
 ('of', 0.08703183401054254),
 ('office', 0.07398253677732308),
 ('representatives', 0.07101160758010638),
 ('education', 0.060941680540831766),
 ('dept', 0.05721520209005032),
 ('senate', 0.054725604828595474),
 ('affairs', 0.042325234201728526),
 ('secretary', 0.034887032243401304),
 ('public', 0.03417429858113247),
 ('embassy', 0.03329322574782357)]